# Create Task
first we need to create a task from coroutines then run tasks with asyncio.run


In [ ]:
import asyncio

async def delay(delay_seconds: int, task_number: int) -> int:
 print(f'sleeping for {delay_seconds} second(s)- {task_number}')
 await asyncio.sleep(delay_seconds)
 print(f'finished sleeping for {delay_seconds} second(s)- {task_number}')
 return delay_seconds

async def main():
 task_1 = asyncio.create_task(delay(3,1))
 task_2 = asyncio.create_task(delay(3,2))
 task_3 = asyncio.create_task(delay(3,3))
 await task_1
 await task_2
 await task_3

asyncio.run(main())



# Image
<img src="PdfImage.png" height="500">


# Cancelling a task
Each task object has a method named cancel,
which we can call whenever we’d like to stop a task. Canceling a task will cause that
task to raise a CancelledError when we await it, which we can then handle as
needed.

In [ ]:
import asyncio

import asyncio
from asyncio import CancelledError

async def delay(delay_seconds: int, task_number: int=0) -> int:
   print(f'sleeping for {delay_seconds} second(s)- {task_number}')
   await asyncio.sleep(delay_seconds)
   print(f'finished sleeping for {delay_seconds} second(s)- {task_number}')
   return delay_seconds

async def main():
   long_task = asyncio.create_task(delay(10))
   seconds_elapsed = 0
   while not long_task.done():
      print('Task not finished, checking again in a second.')
      await asyncio.sleep(1)
      seconds_elapsed = seconds_elapsed + 1
      if seconds_elapsed == 5:
       long_task.cancel()
   try:
      await long_task
   except CancelledError:
      print('Our task was cancelled')
asyncio.run(main())




# Setting a timeout and canceling with wait_for
Checking every second or at some other time interval, and then canceling a task, as we
did in the previous example, isn’t the easiest way to handle a timeout. Ideally, we’d
have a helper function that would allow us to specify this timeout and handle cancella￾tion for us.
 asyncio provides this functionality through a function called asyncio.wait_for.
This function takes in a coroutine or task object, and a timeout specified in seconds. It
then returns a coroutine that we can await. If the task takes more time to complete
than the timeout we gave it, a TimeoutException will be raised. O

In [ ]:
import asyncio
async def main():
    delay_task = asyncio.create_task(delay(2))
    try:
        result = await asyncio.wait_for(delay_task, timeout=1)
        print(result)
    except asyncio.exceptions.TimeoutError:
        print('Got a timeout!')
        print(f'Was the task cancelled? {delay_task.cancelled()}')
asyncio.run(main())

Canceling tasks automatically if they take longer than expected is normally a good
idea. Otherwise, we may have a coroutine waiting indefinitely, taking up resources
that may never be released. However, in certain circumstances we may want to keep
our coroutine running. For example, we may want to inform a user that something is
taking longer than expected after a certain amount of time but not cancel the task
when the timeout is exceeded.
 To do this we can wrap our task with the asyncio.shield function. This function
will prevent cancellation of the coroutine we pass in, giving it a “shield,” which cancel￾lation requests then ignore.

In [ ]:
async def main():
    task = asyncio.create_task(delay(15))
    try:
        result = await asyncio.wait_for(asyncio.shield(task), 5)
        print(result)
    except TimeoutError:
        print("Task took longer than five seconds, it will finish soon!")
        result = await task
        print(result)